In [2]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


In [23]:
%%writefile app.py
import streamlit as st
import pandas as pd
from datetime import datetime
import os

st.set_page_config(page_title="💰 Payment Tracker", layout="wide")

DATA_FILE = "data.csv"

# ----- Load existing data -----
if os.path.exists(DATA_FILE):
    df = pd.read_csv(DATA_FILE)
else:
    df = pd.DataFrame(columns=[
        "Payment Type", "Vendor", "Name", "Country Code", "Exchange Rate",
        "Total Amount (USD)", "Total Amount (LKR)", "Paid Amount (USD)",
        "Paid Amount (LKR)", "Due Amount (USD)", "Due Amount (LKR)",
        "Due Date", "Invoiced Date", "Payment Terms", "Payment Date", "Payment Status"
    ])

st.session_state.payments = df

# ----- Custom CSS with background image and enhancements -----
st.markdown("""
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;500;700&display=swap');

        html, body, [class*="css"] {
            font-family: 'Inter', sans-serif;
        }

        .stApp {
            background: url("https://www.shutterstock.com/image-photo/stack-silver-coins-trading-chart-financial-2223000619") no-repeat center center fixed;
            background-size: cover;
        }

        .metric-box {
            background: rgba(255, 255, 255, 0.85);
            border-left: 8px solid #2196F3;
            padding: 20px;
            border-radius: 15px;
            margin-bottom: 10px;
            box-shadow: 0 4px 10px rgba(0,0,0,0.08);
            transition: 0.3s ease;
        }

        .metric-box:hover {
            transform: scale(1.02);
        }

        .metric-box h3 {
            font-size: 16px;
            margin-bottom: 5px;
            color: #444;
        }

        .metric-box p {
            font-size: 22px;
            font-weight: bold;
            color: #0b5394;
        }

        .stButton>button {
            background: linear-gradient(to right, #4CAF50, #81C784);
            color: white;
            padding: 0.6em 1.2em;
            border: none;
            border-radius: 8px;
            font-weight: 600;
            transition: all 0.3s ease;
        }

        .stButton>button:hover {
            transform: scale(1.03);
        }

        .stDownloadButton>button {
            background: linear-gradient(to right, #2196F3, #64B5F6);
            color: white;
            border-radius: 8px;
            font-weight: 600;
            padding: 0.6em 1.2em;
        }

        .block-container {
            background: rgba(255,255,255,0.92);
            border-radius: 20px;
            padding: 2rem;
        }

        .stDataFrame tbody tr:nth-child(even) {
            background-color: #f9f9f9;
        }

        .stDataFrame tbody tr:hover {
            background-color: #e0f7fa;
        }
    </style>
""", unsafe_allow_html=True)

# ----- Page Header -----
st.markdown("## 💼 Payment Tracker Dashboard")
st.markdown("Efficiently monitor all your payments with real-time calculations and export options.")

# ----- Dashboard Metrics -----
df = st.session_state.payments
total_lkr = df["Total Amount (LKR)"].sum()
paid_lkr = df["Paid Amount (LKR)"].sum()
due_lkr = df["Due Amount (LKR)"].sum()

st.subheader("📊 Overview")
col1, col2, col3 = st.columns(3)
with col1:
    st.markdown(f'''
        <div class="metric-box">
            <h3>Total Amount (LKR)</h3>
            <p>{total_lkr:,.2f}</p>
        </div>
    ''', unsafe_allow_html=True)
with col2:
    st.markdown(f'''
        <div class="metric-box">
            <h3>Total Paid (LKR)</h3>
            <p>{paid_lkr:,.2f}</p>
        </div>
    ''', unsafe_allow_html=True)
with col3:
    st.markdown(f'''
        <div class="metric-box">
            <h3>Total Due (LKR)</h3>
            <p>{due_lkr:,.2f}</p>
        </div>
    ''', unsafe_allow_html=True)

# ----- Payment Entry Form -----
st.markdown("### 📝 Add New Payment Record")
with st.form("payment_form"):
    payment_type = st.selectbox("Payment Type", ["Usual Payment", "Advance Payment"])
    vendor = st.text_input("Vendor Name")
    name = st.text_input("Payment Description / Name")
    country_code = st.text_input("Currency Code (e.g., USD, EUR)", max_chars=3, value="USD").upper()
    exchange_rate = st.number_input("Exchange Rate to LKR", min_value=0.0, step=0.01)

    total_amount_usd = st.number_input("Total Amount (USD)", min_value=0.0, step=0.01)
    paid_amount_usd = st.number_input("Paid Amount (USD)", min_value=0.0, step=0.01)
    due_amount_usd = total_amount_usd - paid_amount_usd

    total_amount_lkr = total_amount_usd * exchange_rate
    paid_amount_lkr = paid_amount_usd * exchange_rate
    due_amount_lkr = due_amount_usd * exchange_rate

    st.write(f"**Due Amount in USD:** {due_amount_usd:,.2f}")
    st.write(f"**Due Amount in LKR:** {due_amount_lkr:,.2f}")

    due_date = st.date_input("Due Date", value=datetime.today())
    invoiced_date = st.date_input("Invoiced Date", value=datetime.today())
    payment_terms = st.text_input("Payment Terms (e.g., Net 30)")
    payment_date = st.date_input("Payment Date", value=datetime.today())
    payment_status = st.selectbox("Payment Status", ["Pending", "Completed"])

    submitted = st.form_submit_button("Add Payment")

    if submitted and name and vendor:
        new_row = {
            "Payment Type": payment_type,
            "Vendor": vendor,
            "Name": name,
            "Country Code": country_code,
            "Exchange Rate": exchange_rate,
            "Total Amount (USD)": total_amount_usd,
            "Total Amount (LKR)": total_amount_lkr,
            "Paid Amount (USD)": paid_amount_usd,
            "Paid Amount (LKR)": paid_amount_lkr,
            "Due Amount (USD)": due_amount_usd,
            "Due Amount (LKR)": due_amount_lkr,
            "Due Date": due_date.strftime("%Y-%m-%d"),
            "Invoiced Date": invoiced_date.strftime("%Y-%m-%d"),
            "Payment Terms": payment_terms,
            "Payment Date": payment_date.strftime("%Y-%m-%d"),
            "Payment Status": payment_status
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(DATA_FILE, index=False)
        st.session_state.payments = df
        st.success("✅ Payment added successfully.")

# ----- Display Table -----
st.markdown("### 📑 Payment Records Table")
st.dataframe(st.session_state.payments, use_container_width=True)

# ----- Download Button -----
if not df.empty:
    csv = df.to_csv(index=False).encode("utf-8")
    st.download_button(
        label="📥 Download CSV",
        data=csv,
        file_name="payment_records.csv",
        mime="text/csv"
    )


Overwriting app.py


In [24]:
from pyngrok import ngrok
import os
import time

# 🛑 Kill any existing ngrok tunnels
ngrok.kill()

# 🔑 Set your Ngrok auth token directly (Replace this with your actual token)
NGROK_AUTH_TOKEN = "30JodevOaUgC5nAPXDqA8FKxnyb_6UrC9pQWddGw7kp3r89cy"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 🚀 Launch the Streamlit app in the background
os.system("streamlit run app.py &")
time.sleep(5)  # Allow Streamlit to start

# 🌐 Connect ngrok to the running Streamlit app
public_url = ngrok.connect(8501)

# 📢 Display the public URL
print("🌍 Your app is live at:", public_url)


🌍 Your app is live at: NgrokTunnel: "https://c83a74147c1f.ngrok-free.app" -> "http://localhost:8501"
